In [2]:
from transformers import AutoProcessor, AutoModelForCausalLM  
from PIL import Image
import requests
import copy
import torch
%matplotlib inline  

c:\Users\Administrator\Desktop\Call-Audio-kush\callenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype='auto').eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

c:\Users\Administrator\Desktop\Call-Audio-kush\callenv\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [4]:
def run_example(task_prompt, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt").to('cuda', torch.float16)
    generated_ids = model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=1024,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text, 
        task=task_prompt, 
        image_size=(image.width, image.height)
    )

    return parsed_answer


In [14]:
path = 'better images/17.jpg'
image = Image.open(path)
# Cell 5: Perform OCR
task_prompt = '<OCR>'
ocr_result = run_example(task_prompt)

In [20]:
ocr_text=ocr_result['<OCR>']

In [21]:
sys_prompt = '''Extract the following key information from the OCR result below:
- Batch No. (or Lot No. or B No.)
- MFG Date (Manufacturing Date)
- Exp Date (Expiry Date)
- Price (M.R.P.)
'''
prompt = sys_prompt + '\n' + ocr_text
prompt

'Extract the following key information from the OCR result below:\n- Batch No. (or Lot No. or B No.)\n- MFG Date (Manufacturing Date)\n- Exp Date (Expiry Date)\n- Price (M.R.P.)\n\nBETA\nLETS\nOLOL\nBatch No. :\nMfg. Date :\nB7AFX011\n07/2024\nExpiry Date:\n06/2026\nM.R.P.₹\n70.30\n'

In [24]:
import requests
OLLAMA_URL = "http://localhost:11434/api/generate"

def query_gemma(prompt, model="gemma3:1b"):
    payload = {
        "model":model,
        "prompt":prompt,
        "stream":False
    }
    response = requests.post(OLLAMA_URL, json=payload)
    response.raise_for_status()
    return response.json()["response"]

if __name__=="__main__":
    prompt=prompt
    result=query_gemma(prompt)
    print(result)

Here’s the extracted information from the OCR result:

*   **Batch No.:** B7AFX011
*   **Mfg. Date:** 07/2024
*   **Expiry Date:** 06/2026
*   **M.R.P.₹:** 70.30
